# Scrape Online Travel Agent (OTA) Property Prices

@Nileka add a description here as to what this notebook offers

### Execute the cell below once to turn off debugging

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

## Logging Configuration
___TODO___ add logging of status

In [2]:
import logging


## Initialize the OTA scrape class

In [2]:
import sys
sys.path.insert(1, '../../lib/wrangler')
import otaWebScraper as otaws
import datetime

dataDirPath="../../data/hospitality/bookings/scraper"

if debug:
    import importlib
    otaws = importlib.reload(otaws)
    
clsScraper = otaws.OTAWebScraper()

All packages in OTAWebScraper loaded successfully!
All packages in OTAWebScraper loaded successfully!
Initialing OTAWebScraper class for  data


## Initialize the set of OTA URL for Scraping

In [3]:
from datetime import date

path = dataDirPath
file = "otaInputURLs.json"
start_date = date(2022,9,11)
end_date = date(2022,9,12)

try:
    if start_date < date.today():
        raise ValueError("Start date must be greater than today: %s" % str(date.today()))
    if end_date <= start_date:
        raise ValueError("End date %s is invalid. It must be greater than Start Date: %s" % (str(end_date),str(start_date)))
    scrape_crietia_dict = {"pageOffset":10,
                           "pageUpperLimit":550,
                           "startDate": start_date,
                           "endDate" : end_date,
             }
    _, _ota_url_parameterized_list  = clsScraper.build_scrape_url_list(dirPath=path,
                                                                       fileName=file,
                                                                       **scrape_crietia_dict)
except Exception as err:
    _s_fn_id = "Class <WebScraper> Function <read_folder_csv_to_df>"
    print("[Error]"+_s_fn_id, err)
    print(traceback.format_exc())

#print(_ota_url_parameterized_list)
print("Completed parameterizing urls with %d instances." % (len(_ota_url_parameterized_list)))

Loaded 1 properties to begin scraping OTA data.
Processing booking.com ...
Completed parameterizing urls with 784 instances.


In [4]:
#path = dataDirPath
file = "otaInputURLs.json"
property_dict = clsScraper.load_ota_list(dirPath=dataDirPath, fileName=file)
_scrape_tags_df = clsScraper.get_scrape_html_tags(property_dict)
print(_scrape_tags_df)

        variable   tag                   code          ota
0  content_block   div            .d20f4628d0  booking.com
0  property_name  span  fcab3ed991 a23c043802  booking.com
0      room_type  span             df597226dd  booking.com
0      room_rate   div  fcab3ed991 bd73d13072  booking.com
0   review_score   div  b5cd09854e d10a6220b4  booking.com
0       location   div             a1fbd102d9  booking.com
0          Other   div             d22a7c133b  booking.com


## Prepare folder & file structure

In [5]:
from datetime import datetime, timezone

dirPath = None
_search_dt = datetime.now()
kwargs = {'search_datetime': _search_dt}
''' include the timezone '''
_search_dt = (_search_dt.replace(tzinfo=timezone.utc)).isoformat()

_current_search_data_dir = clsScraper.get_search_data_dir_path(dirPath, **kwargs)
print("Extracting data into %s for search datetime: %s" % (_current_search_data_dir,str(_search_dt)))

Extracting data into ../../data/hospitality/bookings/scraper/rates/2022-9-11-13-30/ for search datetime: 2022-09-11T13:51:11.517138+00:00


## Scrape data into CSVs
___TODO:___ hault if internet connection times out; might be possible with airflow.

In [6]:
_l_saved_files = clsScraper.scrape_url_list(_ota_url_parameterized_list, _search_dt, _current_search_data_dir)
print("Scraping completed and data saved in %s!" % _current_search_data_dir)

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> 'NoneType' object has no attribute 'text'
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hospitality/../../lib/wrangler/otaWebScraper.py", line 576, in _scrape_bookings_to_csv
    _data_dict['review_score'] = _list.find('div', class_='b5cd09854e d10a6220b4').text
AttributeError: 'NoneType' object has no attribute 'text'

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> 'NoneType' object has no attribute 'text'
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hospitality/../../lib/wrangler/otaWebScraper.py", line 576, in _scrape_bookings_to_csv
    _data_dict['review_score'] = _list.find('div', class_='b5cd09854e d10a6220b4').text
AttributeError: 'NoneType' object has no attribute 'text'

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> 'NoneType' object has no attribute 'text'
Traceback (most recent call last):
  File "/home/nuwan/wo

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-11&checkout=2022-09-12&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=290
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hospitality/../../lib/wrangler/otaWebScraper.py", line 567, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-11&checkout=2022-09-12&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=360
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hospitality/../../lib/wrangler/otaWebScraper.py", line 567, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> HTTPSConnectionPool(host='www.booking.com', port=443): Max retries exceeded with url: /searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-11&checkout=2022-09-12&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=430 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f5556933c10>: Failed to establish a new connection: [Errno -2] Name or service not known'))
Traceback (most recent call last):
  File "/home/nuwan/.local/lib/python3.8/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/nuwan/.local/lib/python3.8/site-packages/urllib3/util/connection.

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20079110&dest_type=city&checkin=2022-09-11&checkout=2022-09-12&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=470
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hospitality/../../lib/wrangler/otaWebScraper.py", line 567, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36

...

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-11&checkout=2022-09-12&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=260
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hospitality/../../lib/wrangler/otaWebScraper.py", line 567, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-11&checkout=2022-09-12&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=330
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hospitality/../../lib/wrangler/otaWebScraper.py", line 567, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-11&checkout=2022-09-12&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=400
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hospitality/../../lib/wrangler/otaWebScraper.py", line 567, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-11&checkout=2022-09-12&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=470
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hospitality/../../lib/wrangler/otaWebScraper.py", line 567, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36

[Error]Class <WebScraper> Function <_scrape_bookings_to_csv> No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36e124997ccdbec4823b6c98d5931c2e&aid=304142&lang=en-gb&sb=1&src_elem=sb&src=searchresults&dest_id=20021296&dest_type=city&checkin=2022-09-11&checkout=2022-09-12&group_adults=1&no_rooms=1&group_children=0&selected_currency=USD&offset=540
Traceback (most recent call last):
  File "/home/nuwan/workspace/rezgate/notebooks/hospitality/../../lib/wrangler/otaWebScraper.py", line 567, in _scrape_bookings_to_csv
    raise ValueError("No data received for %s" % (url))
ValueError: No data received for https://www.booking.com/searchresults.en-gb.html?ss=Las+Vegas&label=gen173nr-1DCAEoggI46AdIM1gEaIUBiAEBmAEJuAEXyAEM2AED6AEBiAIBqAIDuAK_g7aYBsACAdICJDFiNWFiNzM3LTQ0YmItNDIzOC04NDM0LWRjMDFlNmZhYTUwM9gCBOACAQ&sid=36

## Use Spark to Load Data into Database

### Declare sparkworkload class

In [101]:
import sys
sys.path.insert(1, '../../lib/wrangler')
import sparkWorkLoads as spark
import datetime

if debug:
    import importlib
    spark = importlib.reload(spark)

clsSparkWL = spark.SparkWorkLoads()
#_session = clsSparkWL.get_spark_session()

All packages in SparkWorkLoads loaded successfully!
Connection complete! ready to load data.


### Read CSV into spark dataframe

In [62]:
dirPath = _current_search_data_dir
#dirPath = "../../data/hospitality/bookings/scraper/rates/2022-9-7-9-0/"
_search_sdf = clsSparkWL.read_csv_to_sdf(filesPath=dirPath)
_search_sdf = _search_sdf.distinct()

### Extract room rate decimal value

In [63]:
from pyspark.sql.functions import substring,lit,col
from pyspark.sql.types import StringType,BooleanType,DateType,DecimalType,FloatType, IntegerType,LongType, ShortType, TimestampType

_search_sdf=_search_sdf.withColumn("currency", lit("US$"))
_search_sdf=_search_sdf.withColumn('room_rate', substring('room_rate', 4,10))

''' reset data types to match table '''
_search_sdf = _search_sdf.withColumn("destination_id",col("destination_id").cast(StringType())) \
    .withColumn("room_rate",col("room_rate").cast(FloatType()))
#    .withColumn("search_datetime",col("search_datetime").cast(DateType()))

_search_sdf.printSchema()
#_search_sdf.show(n=2, vertical=True, truncate=False)
print("Split and Extraction complete!")

root
 |-- search_dt: timestamp (nullable = true)
 |-- checkin_date: timestamp (nullable = true)
 |-- property_name: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- room_rate: float (nullable = true)
 |-- review_score: double (nullable = true)
 |-- destination_id: string (nullable = true)
 |-- location_desc: string (nullable = true)
 |-- other_info: string (nullable = true)
 |-- currency: string (nullable = false)



-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 search_dt      | 2022-09-11 21:51:11.517138                                                                                                                                                            
 checkin_date   | 2022-09-11 00:00:00                                                                                                                                                                   
 property_name  | Maison Pierre Lafitte                                                                                                                                                                 
 room_type      | One-Bedroom Flat                                                                                                                                                                  

## Cleanup and Save data to table

In [64]:
''' Get destination id dictionary '''
destfilesPath = "../../data/hospitality/bookings/scraper/destinations"
destinations_sdf = clsSparkWL.read_csv_to_sdf(filesPath=destfilesPath)
destinations_sdf = destinations_sdf.selectExpr("city as destination_name", \
                                               "destinationID as destination_id")
destinations_sdf = destinations_sdf.withColumn("destination_name",col("destination_name").cast(StringType())) \
                                .withColumn("destination_id",col("destination_id").cast(StringType()))

destinations_sdf.printSchema()
#destinations_sdf.show(n=2, vertical=True, truncate=False)
print("Destination dictionary loarded!")

''' Lookup & augment destination name '''
aug_search_sdf = destinations_sdf.join(_search_sdf,"destination_id")
aug_search_sdf.show(n=2, vertical=True, truncate=False)
print("Destination names augmented to dataframe!")

''' Save dataframe to table '''
_s_tbl_name = "ota_property_prices"
count = clsSparkWL.insert_sdf_into_table(save_sdf=aug_search_sdf, dbTable=_s_tbl_name)
print("%d Data saved to %s" % (count,_s_tbl_name))

root
 |-- destination_name: string (nullable = true)
 |-- destination_id: string (nullable = true)

-RECORD 0-------------------------
 destination_name | Las Vegas     
 destination_id   | 20079110      
-RECORD 1-------------------------
 destination_name | New York City 
 destination_id   | 20088325      
only showing top 2 rows



## Read data from table

In [103]:
_s_tbl_name = "ota_property_prices"
data = clsSparkWL.get_data_from_table(dbTable=_s_tbl_name)
print("%d records loaded from %s" %(data.count(),_s_tbl_name))

Wait a moment, retrieving data ...
Loading complete!
-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 uuid             | null                                                                                                                                                                                  
 ota_name         | null                                                                                                                                                                                  
 search_dt        | 2022-09-11 21:51:11.517138                                                                                                                                                            
 property_name    | Maison Pierre Lafitte                                                                                              